In [1]:
!pip install tqdm

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandapower as pp
import pandapower.networks as pn
from pandapower.estimation import estimate, remove_bad_data, chi2_analysis
import pandas as pd
import numpy as np
# from tqdm import tqdm

from scipy import stats

print(pp.__version__)



2.0.1


In [2]:
def create_measurement_unit(df_measurement, net):
    list_value=[]
    list_std=[]
#     print(net)
    for index, row in df_measurement.iterrows():
        if row['element_type'] =='bus':
            if row['meas_type'] =='v':
                mu= net.res_bus.iloc[row['element'],0]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
            elif row['meas_type'] =='p':
                mu= net.res_bus.iloc[row['element'],2]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4
            elif row['meas_type'] =='q':
                mu= net.res_bus.iloc[row['element'],3]
                sigma = (abs(mu)*1.03-abs(mu)*0.97)/4

        elif row['element_type'] =='line':

            if row['side']=='from':
                if row['meas_type'] =='p':
                    mu= net.res_line.iloc[row['element'],0]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type'] =='q':
                    mu = net.res_line.iloc[row['element'],1]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type']=='i':
                    mu=net.res_line.iloc[row['element'],6]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
            elif row['side']=='to':
                pass

        elif row['element_type'] =='trafo':
            'create trafo measurement'
            if row['side']=='from':
                pass
            elif row['side']=='to':
                if row['meas_type'] =='v':
                    mu =net.res_trafo.iloc[row['element'],10]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        #                 print(mu)
                elif row['meas_type'] =='p':
                    mu =net.res_trafo.iloc[row['element'],2]
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
                elif row['meas_type'] =='q':
                    mu =net.res_trafo.iloc[row['element'],3]
        #                 print(mu)
                    sigma = (abs(mu)*1.05-abs(mu)*0.95)/4
        #     print('Mean:',mu, 'standard deviation:', sigma)
        value = np.random.normal(mu, sigma, 1)
        #     print('Random value:', value[0])
        list_value.append(value[0])
        list_std.append(sigma)
#     print(list_value)
    df_measurement['value']=list_value
    df_measurement['std_dev']=list_std

    for index, row in df_measurement.iterrows():
        if row['element_type']=='bus':
#             print('create bus measurement')
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'])

        elif row['element_type']=='line':
#             print('create line measurement')
            pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                  std_dev=row['std_dev'], element=row['element'], side=row['side'])

        elif row['element_type']=='trafo':
#             print('create trafo measurement')
            if row['meas_type'] in ['p','q']:
                pp.create_measurement(net, row['meas_type'], row['element_type'], value=row['value'], 
                                      std_dev=row['std_dev'], element=row['element'], side=row['side'])
    return df_measurement, net

# NO solar PV

In [13]:
net1 = pn.simple_four_bus_system()
net1.sgen['p_mw']=[0,0]
net1.sgen['q_mvar']=[0,0]
net1.sgen['sn_mva']=[0,0]
pp.runpp(net1)

In [14]:
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']
df_measurement, net1 = create_measurement_unit(df_measurement, net1)
success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net1.res_bus_est.vm_pu, net1.res_bus_est.va_degree

In [15]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=[0]
participation_bus=list(set(list(net1.bus.index))-set(list_slack_bus+list_zero_inject_bus))

In [16]:
df_LPF=pd.DataFrame()
df_LPF['name']=net1.bus['name'].copy()
df_LPF['name']=net1.bus['name'].copy()
known_generation=-net1.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net1.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net1.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net1.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net1.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -8.55054064773213 %


In [17]:
line_loss

0.016081240355399776

In [18]:
net1.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.074774,0.021728,-0.062258,-0.020114,0.012516,0.001614,0.114003,0.114005,0.114005,0.985858,-150.609578,0.828337,-148.911306,80.285399
1,0.033439,0.010054,-0.029874,-0.009596,0.003565,0.000458,0.060844,0.060846,0.060846,0.828337,-148.911306,0.744328,-147.842533,42.849282


In [23]:
line_loss_participation

0.25056260679799175

In [19]:
net1.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.986974,0.000000,-0.064181,0.012512
1,0.985858,-150.609578,-0.011386,-0.034904
2,0.828337,-148.911306,0.028819,0.010060
3,0.744328,-147.842533,0.029874,0.009596


In [20]:
line_loss

0.016081240355399776

In [21]:
line_loss_participation

0.25056260679799175

In [22]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,-0.177408
2,bus3,0.028819,0.010060,0.449026
3,bus4,0.029874,0.009596,0.465469


In [171]:
list_LPF_zero_injection_bus=[]
for round in range(0,100):
# for round in tqdm(range(0,100)):
#     print(round)
    net1 = pn.simple_four_bus_system()
    net1.sgen['p_mw']=[0,0]
    net1.sgen['q_mvar']=[0,0]
    net1.sgen['sn_mva']=[0,0]
    pp.runpp(net1)
    df_measurement=pd.DataFrame()
    df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
    df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
    df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
    df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']
    df_measurement, net1 = create_measurement_unit(df_measurement, net1)
    success = estimate(net1, init="flat", calculate_voltage_angles=True, zero_injection=[1])
#     V_est, delta_est = net1.res_bus_est.vm_pu, net1.res_bus_est.va_degree
    list_slack_bus=[0]
    list_zero_inject_bus=[1]
    list_known_generation_bus=[0]
    participation_bus=list(set(list(net1.bus.index))-set(list_slack_bus+list_zero_inject_bus))
    df_LPF=pd.DataFrame()
    df_LPF['name']=net1.bus['name'].copy()
    known_generation=-net1.res_bus_est.iloc[list_slack_bus[0],2]

    allocate_p_MW=[]
    allocate_q_MVAR=[]
    for index, row in df_LPF.iterrows():
        if index in list_known_generation_bus+list_zero_inject_bus:
            allocate_p_MW.append(0)
            allocate_q_MVAR.append(0)
        else:
            allocate_p_MW.append(net1.res_bus_est.iloc[index,2])
            allocate_q_MVAR.append(net1.res_bus_est.iloc[index,3])
    df_LPF['allocate_p_MW']=allocate_p_MW
    df_LPF['allocate_1_MVAR']=allocate_q_MVAR


    df_LPF['LPF']=net1.res_bus_est['p_mw']/known_generation

    loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

#     print('loss is:', loss*100, '%')
    list_LPF_zero_injection_bus.append(df_LPF.LPF[1])

In [172]:
stats.trim_mean(np.array(list_LPF_zero_injection_bus), 0.1)

-0.09138461488661316

In [173]:
np.array(list_LPF_zero_injection_bus).mean()

0.34064070238724187

# integration solar PV

In [29]:
net2 = pn.simple_four_bus_system()
pp.runpp(net2)
V, delta=net2.res_bus.vm_pu, net2.res_bus.va_degree
# print(net)

In [30]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net2 = create_measurement_unit(df_measurement, net2)

success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])
V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree


In [36]:
net2.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.027611,0.013328,-0.025713,-0.013088,0.001899,0.000241,0.044405,0.044408,0.044408,0.996608,-0.208127,0.93776,0.992541,31.273099
1,0.015713,0.008088,-0.015000,-0.008000,0.000713,0.000088,0.027200,0.027203,0.027203,0.937760,0.992541,0.90200,1.815913,19.157250


In [31]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net2.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

In [32]:
df_LPF=pd.DataFrame()
df_LPF['name']=net2.bus['name'].copy()
df_LPF['name']=net2.bus['name'].copy()
known_generation=-net2.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net2.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net2.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net2.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net2.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -235.049741894619 %


In [38]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,2.570153
2,bus3,0.010767,0.005032,-0.566269
3,bus4,0.014911,0.007891,-0.784228


In [71]:
net2.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.029256,0.013485,-0.027224,-0.013227,0.002033,0.000258,0.045940,0.045943,0.045943,1.012140,-150.215436,0.950887,-149.056207,32.354131
1,0.015549,0.008101,-0.014867,-0.008017,0.000682,0.000084,0.026613,0.026616,0.026616,0.950887,-149.056207,0.915961,-148.252009,18.743916


In [34]:
net2.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.982694,0.000000,0.019014,0.137131
1,1.004942,-150.210953,-0.048869,-0.153416
2,0.945200,-149.049534,0.010767,0.005032
3,0.909985,-148.252843,0.014911,0.007891


In [39]:
line_loss_participation

-0.13835085880521297

In [37]:

line_loss

0.002630583809100605

In [174]:
list_LPF_zero_injection_bus=[]
for round in range(0,100):
# for round in tqdm(range(0,100)):
#     print(round)
    net2 = pn.simple_four_bus_system()
    pp.runpp(net2)
    V, delta=net2.res_bus.vm_pu, net2.res_bus.va_degree
    # print(net)
    df_measurement=pd.DataFrame()
    df_measurement=pd.DataFrame()
    df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
    df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
    df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
    df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

    #where bus measurement has 3%error
    #where line measurement has 5%error
    #where trafo measurement has 5%error

    df_measurement, net2 = create_measurement_unit(df_measurement, net2)

    success = estimate(net2, init="flat", calculate_voltage_angles=True, zero_injection=[1])
    V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

    list_slack_bus=[0]
    list_zero_inject_bus=[1]
    list_known_generation_bus=list(set(list_slack_bus))
    participation_bus=list(set(list(net2.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))
    df_LPF=pd.DataFrame()
    df_LPF['name']=net2.bus['name'].copy()
    known_generation=-net2.res_bus_est.iloc[list_slack_bus[0],2]

    allocate_p_MW=[]
    allocate_q_MVAR=[]
    for index, row in df_LPF.iterrows():
        if index in list_known_generation_bus+list_zero_inject_bus:
            allocate_p_MW.append(0)
            allocate_q_MVAR.append(0)
        else:
            allocate_p_MW.append(net2.res_bus_est.iloc[index,2])
            allocate_q_MVAR.append(net2.res_bus_est.iloc[index,3])
    df_LPF['allocate_p_MW']=allocate_p_MW
    df_LPF['allocate_q_MVAR']=allocate_q_MVAR


    df_LPF['LPF']=net2.res_bus_est['p_mw']/known_generation

    loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

    # print('loss is:', loss*100, '%')
    list_LPF_zero_injection_bus.append(df_LPF.LPF[1])

In [175]:
stats.trim_mean(np.array(list_LPF_zero_injection_bus), 0.1)

0.23873394210645643

In [176]:
df_LPF

,name,allocate_p_MW,allocate_q_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,6.851735
2,bus3,0.010696,0.005426,-2.104078
3,bus4,0.015251,0.008021,-2.999953


# Solar PV at bus 2

In [40]:
net3 = pn.simple_four_bus_system()
net3.sgen['p_mw']=[0.020,0]
net3.sgen['q_mvar']=[0.005,0]
net3.sgen['sn_mva']=[0.03,0]
pp.runpp(net3)

In [41]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net3 = create_measurement_unit(df_measurement, net3)

success = estimate(net3, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

In [42]:
success

True

In [43]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net3.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

df_LPF=pd.DataFrame()
df_LPF['name']=net3.bus['name'].copy()
known_generation=-net3.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net3.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net3.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net3.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net3.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: 160.88255368162535 %


In [44]:
print(df_LPF)

      name  allocate_p_MW  allocate_1_MVAR       LPF
0  bus1ref       0.000000         0.000000 -1.000000
1     bus2       0.000000         0.000000 -2.196509
2     bus3       0.012909         0.005174  0.792573
3     bus4       0.029583         0.009670  1.816252


In [45]:
net3.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.051024,0.015938,-0.045351,-0.01521,0.005673,0.000729,0.076753,0.076755,0.076755,1.005268,-150.398857,0.899517,-149.210901,54.052681
1,0.032442,0.010036,-0.029583,-0.00967,0.002860,0.000366,0.054491,0.054493,0.054493,0.899517,-149.210901,0.824368,-148.304942,38.375380


In [46]:
known_generation

0.01628771936151705

In [47]:
net3.res_bus_est.iloc[list_slack_bus[0],2]

-0.01628771936151705

In [48]:
net3.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,0.991593,0.000000,-0.016288,0.093712
1,1.005268,-150.398857,-0.035776,-0.111053
2,0.899517,-149.210901,0.012909,0.005174
3,0.824368,-148.304942,0.029583,0.009670


In [49]:
sum(net3.res_line_est['pl_mw'])

0.008532679038079978

In [50]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,-2.196509
2,bus3,0.012909,0.005174,0.792573
3,bus4,0.029583,0.009670,1.816252


In [51]:
line_loss

0.008532679038079978

# Solar PV panel at bus 3

In [66]:
net4 = pn.simple_four_bus_system()
# print(net4.sgen)
net4.sgen['p_mw']=[0,0.015]
net4.sgen['q_mvar']=[0,0.002]
net4.sgen['sn_mva']=[0,0.02]
print(net4.sgen)

pp.runpp(net4)

  name  bus   p_mw  q_mvar  sn_mva  scaling  in_service  type  current_source
0  pv1    2  0.000   0.000    0.00      1.0        True  None            True
1  pv2    3  0.015   0.002    0.02      1.0        True  None            True


In [67]:
df_measurement=pd.DataFrame()
df_measurement=pd.DataFrame()
df_measurement['meas_type']=['v','v','v','v','p','p','q','q','i','i']
df_measurement['element_type']=['bus','bus','bus','bus','line','line','line','line','line','line']
df_measurement['element']=[0,1,2,3,0,1,0,1,0,1]
df_measurement['side']=['None','None','None','None','from','from','from','from','from','from']

#where bus measurement has 3%error
#where line measurement has 5%error
#where trafo measurement has 5%error

df_measurement, net4 = create_measurement_unit(df_measurement, net4)

success = estimate(net4, init="flat", calculate_voltage_angles=True, zero_injection=[1])
# V_est, delta_est = net2.res_bus_est.vm_pu, net2.res_bus_est.va_degree

In [68]:
list_slack_bus=[0]
list_zero_inject_bus=[1]
list_known_generation_bus=list(set(list_slack_bus))
participation_bus=list(set(list(net3.bus.index))-set(list_known_generation_bus+ list_zero_inject_bus))

df_LPF=pd.DataFrame()
df_LPF['name']=net4.bus['name'].copy()
known_generation=-net4.res_bus_est.iloc[list_slack_bus[0],2]

allocate_p_MW=[]
allocate_q_MVAR=[]
for index, row in df_LPF.iterrows():
    if index in list_known_generation_bus+list_zero_inject_bus:
        allocate_p_MW.append(0)
        allocate_q_MVAR.append(0)
    else:
        allocate_p_MW.append(net4.res_bus_est.iloc[index,2])
        allocate_q_MVAR.append(net4.res_bus_est.iloc[index,3])
df_LPF['allocate_p_MW']=allocate_p_MW
df_LPF['allocate_1_MVAR']=allocate_q_MVAR

          
df_LPF['LPF']=net4.res_bus_est['p_mw']/known_generation

loss=sum(df_LPF.iloc[list_known_generation_bus,:]['LPF'])+sum(df_LPF.iloc[participation_bus,:]['LPF'])

print('loss is:', loss*100, '%')
line_loss=sum(net4.res_line_est['pl_mw'])
line_loss_participation=line_loss/known_generation

loss is: -57.99910022887445 %


In [69]:
df_LPF

,name,allocate_p_MW,allocate_1_MVAR,LPF
0,bus1ref,0.000000,0.000000,-1.000000
1,bus2,0.000000,0.000000,0.488974
2,bus3,0.028485,0.009404,0.272815
3,bus4,0.015368,0.008006,0.147194


In [70]:
net4.res_line_est

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,0.050699,0.018287,-0.044708,-0.017517,0.005990,0.000770,0.078870,0.078872,0.078872,0.986332,-150.404479,0.878723,-148.847941,55.543822
1,0.016223,0.008113,-0.015368,-0.008006,0.000855,0.000107,0.029794,0.029797,0.029797,0.878723,-148.847941,0.839400,-147.910162,20.984075


In [71]:
net4.res_bus_est

,vm_pu,va_degree,p_mw,q_mvar
0,1.018668,0.000000,-0.104410,-0.183243
1,0.986332,-150.404479,0.051054,0.158424
2,0.878723,-148.847941,0.028485,0.009404
3,0.839400,-147.910162,0.015368,0.008006
